# 1. Darknet Clone

Runtime => GPU

In [ ]:
# clone darkenet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15420, done.
remote: Total 15420 (delta 0), reused 0 (delta 0), pack-reused 15420
Receiving objects: 100% (15420/15420), 14.02 MiB | 15.20 MiB/s, done.
Resolving deltas: 100% (10362/10362), done.


In [ ]:
%cd darknet

/content/darknet


In [ ]:
# change makefile to have GPU,OPENCV, CUDNN enabled
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
# Compile Source
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

In [ ]:
# get yolov3 pretrained coco dataset weights
!wget https://pjreddie.com/media/files/yolov3.weights

--2022-06-01 13:20:17--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  20.6MB/s    in 12s     

2022-06-01 13:20:30 (19.2 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



# 2. Tracking

In [ ]:
import cv2
import IPython
import numpy as np
import time
import math
from google.colab.patches import cv2_imshow

In [ ]:
min_confidence = 0.5
weight_file = 'yolov3.weights'
cfg_file = 'cfg/yolov3.cfg'
name_file = 'data/coco.names'
file_name = 'cabc30fc-e7726578.mp4'

In [ ]:
# Load Yolo
net = cv2.dnn.readNet(weight_file, cfg_file)

In [ ]:
classes = [] 
with open(name_file, 'r') as f:
  classes = [line.strip() for line in f.readlines()]
print(classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [ ]:
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]

In [ ]:
# cabc30fc-e7726578.mp4
from google.colab import files 
files.upload()

Saving cabc30fc-e7726578.mp4 to cabc30fc-e7726578.mp4


In [ ]:
def writeFrame(img):
    # use global variable, writer
    global writer
    height, width = img.shape[:2]
    if writer is None and output_name is not None:
        fourcc = cv2.VideoWriter_fourcc(*'MJPG')
        writer = cv2.VideoWriter(output_name, fourcc, 24, (width, height), True)
    if writer is not None:
        writer.write(img)

In [ ]:
frame_count = 0
# initialize the video writer 
writer = None
output_name = 'output_tracking.avi'

detected = False
frame_mode = 'Tracking'
elapsed_time = 0

# csrt : 정확도가 높음
#tracker = cv2.TrackerCSRT_create()
# kcf : 속도 대비 정확도가 좋은 편 - 자주 쓴다
tracker = cv2.TrackerKCF_create() 
# boosting : 속도가 빠르다
# tracker = cv2.TrackerBoosting_create()
# mil
# tracker = cv2.TrackerMIL_create()
# tld
# tracker = cv2.TrackerTLD_create()
# medianflow
# tracker = cv2.TrackerMedianFlow_create()
# mosse
#tracker = cv2.TrackerMOSSE_create()

trackers = cv2.MultiTracker_create()


vs = cv2.VideoCapture(file_name)

while True:
    start_time = time.time()
    frame_count += 1
    ret, frame = vs.read()
    # if frame is None and (frame_count > 5):
    if frame is None and (frame_count > 5):
        print('### No more frame ###')
        break

    if detected:
        frame_mode = 'Tracking'
        (success, boxes) = trackers.update(frame)
        for box in boxes:
           (x, y, w, h) = [int(v) for v in box]
           cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    else:
        frame_mode = 'Detection'
        height, width, channedls = frame.shape
        # Detecting objects
        # https://docs.opencv.org/master/d6/d0f/group__dnn.html
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

        net.setInput(blob)
        outs = net.forward(output_layers)

        class_ids = []
        confidences = []
        boxes = []

        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if (confidence > min_confidence) and (class_id == 2):
                    # Object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)
        # Region of Interest
        roi_left = int(0.3 * width)
        roi_right = int(0.6 * width)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence, 0.4)
        font = cv2.FONT_HERSHEY_COMPLEX
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                # Eliminate Small object(<50)
                if (w > 50) and (x > roi_left) and (x < roi_right):
                    selected = boxes[i]
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 255), 5)    
        trackers.add(tracker, frame, tuple(selected))
        detected = True 

    # cv2_imshow(frame)
    writeFrame(frame)
    frame_time = time.time() - start_time
    elapsed_time += frame_time
    print("[{}] Frame {} time {}".format(frame_mode, frame_count, frame_time))

print("Elapsed time {}".format(elapsed_time))
vs.release()          

[Detection] Frame 1 time 2.9436354637145996
[Tracking] Frame 2 time 0.07193398475646973
[Tracking] Frame 3 time 0.03668808937072754
[Tracking] Frame 4 time 0.04455065727233887
[Tracking] Frame 5 time 0.036234378814697266
[Tracking] Frame 6 time 0.05428576469421387
[Tracking] Frame 7 time 0.03881669044494629
[Tracking] Frame 8 time 0.031688690185546875
[Tracking] Frame 9 time 0.04673910140991211
[Tracking] Frame 10 time 0.040190696716308594
[Tracking] Frame 11 time 0.031239986419677734
[Tracking] Frame 12 time 0.0324251651763916
[Tracking] Frame 13 time 0.06527400016784668
[Tracking] Frame 14 time 0.045333147048950195
[Tracking] Frame 15 time 0.05525541305541992
[Tracking] Frame 16 time 0.05941057205200195
[Tracking] Frame 17 time 0.031029701232910156
[Tracking] Frame 18 time 0.053826093673706055
[Tracking] Frame 19 time 0.07640218734741211
[Tracking] Frame 20 time 0.061081647872924805
[Tracking] Frame 21 time 0.033287763595581055
[Tracking] Frame 22 time 0.03636002540588379
[Tracking] 

In [ ]:
from google.colab import files
files.download(output_name)